In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now, I'll proceed with the plan, starting with installing the necessary libraries.

Now, I will implement the `resize_images` function to preprocess the images.

In [ ]:
import cv2
import numpy as np
import os

def resize_images(input_dir, output_dir, size=(128, 128)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        # Create corresponding directories in the output path
        relative_path = os.path.relpath(root, input_dir)
        output_subdir = os.path.join(output_dir, relative_path)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        for filename in files:
            img_path = os.path.join(root, filename)
            # Check if it's a file and an image (basic check)
            if os.path.isfile(img_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = cv2.imread(img_path)
                if img is not None:
                    original_height, original_width = img.shape[:2]
                    target_width, target_height = size

                    # Calculate the aspect ratio and new dimensions
                    aspect_ratio = original_width / original_height
                    if original_width > original_height:
                        new_width = target_width
                        new_height = int(new_width / aspect_ratio)
                    else:
                        new_height = target_height
                        new_width = int(new_height * aspect_ratio)

                    # Resize the image
                    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

                    # Calculate padding
                    pad_width = target_width - new_width
                    pad_height = target_height - new_height
                    top_pad = pad_height // 2
                    bottom_pad = pad_height - top_pad
                    left_pad = pad_width // 2
                    right_pad = pad_width - left_pad

                    # Add padding (white background)
                    padded_img = cv2.copyMakeBorder(resized_img, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=[255, 255, 255])

                    # Save the resized and padded image to the corresponding output subdirectory
                    output_path = os.path.join(output_subdir, filename)
                    cv2.imwrite(output_path, padded_img)

Next, I will implement the `augment_pictures` function for image augmentation.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil

def augment_pictures(input_dir, output_dir, num_augmented_images_per_image=10):

    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)  # Clear output directory if it exists
    os.makedirs(output_dir, exist_ok=True)

    # Define the ImageDataGenerator with specified augmentations
    datagen = ImageDataGenerator(
        rotation_range=35,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True,
        brightness_range=[0.7, 1.3],
        fill_mode='reflect'
    )

    # Use flow_from_directory to read images from input_dir and generate augmented images
    # We need to manually iterate through subdirectories to control the number of augmented images per original image
    for class_name in os.listdir(input_dir):
        class_input_dir = os.path.join(input_dir, class_name)
        class_output_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_output_dir, exist_ok=True)

        if os.path.isdir(class_input_dir):
            print(f"Augmenting images in {class_input_dir}...")
            # Create a generator for the current class
            class_datagen = ImageDataGenerator(
                rotation_range=35,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.1,
                horizontal_flip=True,
                brightness_range=[0.7, 1.3],
                fill_mode='reflect'
            )

            # Flow from the input directory for the current class
            generator = class_datagen.flow_from_directory(
                input_dir, # Point to the parent directory
                classes=[class_name], # Specify the class to process
                target_size=(128, 128),
                batch_size=1, # Process one image at a time
                save_to_dir=class_output_dir,
                save_prefix='aug',
                save_format='jpeg',
                shuffle=False # Important for processing each image individually
            )

            # Generate augmented images for each image in the class
            num_original_images = len(os.listdir(class_input_dir))
            if num_original_images > 0:
                for i in range(num_original_images * num_augmented_images_per_image):
                     next(generator)
            print(f"Finished augmenting images in {class_input_dir}.")
        else:
             print(f"Not a directory: {class_input_dir}")

Now, I will integrate the resizing and augmentation functions and apply them to the dataset from Google Drive.

In [ ]:
# Define input and output directories
input_dataset_dir = '/content/drive/MyDrive/dataset' # Original dataset from Drive
resized_output_dir = '/content/resized_dataset' # Directory for resized images (local)
augmented_output_dir = '/content/augmented_dataset' # Directory for augmented images (local)

# 1. Resize the images
print(f"Resizing images from {input_dataset_dir} to {resized_output_dir}...")
resize_images(input_dataset_dir, resized_output_dir, size=(128, 128))
print("Resizing complete.")

# 2. Augment the resized images
print(f"Augmenting images from {resized_output_dir} to {augmented_output_dir}...")
augment_pictures(resized_output_dir, augmented_output_dir, num_augmented_images=10)
print("Augmentation complete.")

print(f"Preprocessing and augmentation finished. Resized images saved to {resized_output_dir}, and augmented images saved to {augmented_output_dir}.")

Resizing images from /content/drive/MyDrive/dataset to /content/resized_dataset...


KeyboardInterrupt: 